# Term Project — HDI vs Terrorism (Frequency & Severity)

**Goal:** test whether **Human Development Index (HDI)** is associated with  
1) **terrorism frequency** (how many attacks) and  
2) **terrorism severity** (how many casualties)

using **interpretable ML models** (Linear Regression, Decision Tree, Random Forest) with a **time-aware evaluation**.

---

## Dataset

File: `dsa210project.xlsx` (already cleaned + engineered)

Columns:
- `country`, `region`, `year`
- `hdi`
- `attacks_count`, `total_casualty`
- engineered:
  - `log_attacks = log(attacks_count + 1)`
  - `log_casualties = log(total_casualty + 1)`
  - `hdi_group` (Low / Medium / High / Very High)
  - `casualties_per_attack`, `log_casualties_per_attack`

> We model **log** targets because terrorism data is heavy‑tailed (few extreme years/countries dominate).


In [ ]:
# ===== 0) Imports + load data =====
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

DATA_PATH = r"/mnt/data/dsa210project.xlsx"
df = pd.read_excel(DATA_PATH)

print("Shape:", df.shape)
display(df.head())


In [ ]:
# Quick checks
display(df.isna().sum())
display(df.dtypes)

# Enforce numeric types (defensive)
num_cols = ["year","hdi","attacks_count","total_casualty",
            "log_attacks","log_casualties","casualties_per_attack","log_casualties_per_attack"]
for c in num_cols:
    df[c] = pd.to_numeric(df[c], errors="coerce")

# Drop rows missing critical info
df = df.dropna(subset=["country","region","year","hdi","attacks_count","total_casualty"]).copy()

# Replace infinities if any
for c in ["casualties_per_attack","log_casualties_per_attack"]:
    df[c] = df[c].replace([np.inf, -np.inf], np.nan)

print("After cleaning:", df.shape)


## 1) Exploratory Data Analysis (EDA)

We will check:
- distributions (are there extreme tails?)
- HDI vs outcomes (scatter)
- trend over time (global + region)
- simple correlations (HDI with log targets)


In [ ]:
# 1.1 Distributions
targets = ["attacks_count","total_casualty","log_attacks","log_casualties","log_casualties_per_attack"]

for t in targets:
    plt.figure()
    plt.hist(df[t].dropna(), bins=50)
    plt.xlabel(t)
    plt.ylabel("count")
    plt.title(f"Distribution: {t}")
    plt.show()


In [ ]:
# 1.2 Scatter: HDI vs log targets
pairs = [("hdi","log_attacks"), ("hdi","log_casualties"), ("hdi","log_casualties_per_attack")]

for x,y in pairs:
    plt.figure()
    plt.scatter(df[x], df[y], alpha=0.4)
    plt.xlabel(x)
    plt.ylabel(y)
    plt.title(f"{x} vs {y}")
    plt.show()


In [ ]:
# 1.3 Time trends: global median by year (robust to outliers)
yearly = (df.groupby("year", as_index=False)
            .agg(hdi=("hdi","median"),
                 log_attacks=("log_attacks","median"),
                 log_casualties=("log_casualties","median")))

plt.figure()
plt.plot(yearly["year"], yearly["hdi"])
plt.xlabel("year"); plt.ylabel("median HDI")
plt.title("Global median HDI over time")
plt.show()

plt.figure()
plt.plot(yearly["year"], yearly["log_attacks"])
plt.xlabel("year"); plt.ylabel("median log_attacks")
plt.title("Global median log_attacks over time")
plt.show()

plt.figure()
plt.plot(yearly["year"], yearly["log_casualties"])
plt.xlabel("year"); plt.ylabel("median log_casualties")
plt.title("Global median log_casualties over time")
plt.show()


In [ ]:
# 1.4 Region-year view (median)
region_year = (df.groupby(["region","year"], as_index=False)
                 .agg(hdi=("hdi","median"),
                      log_attacks=("log_attacks","median"),
                      log_casualties=("log_casualties","median")))

plt.figure()
for r in sorted(region_year["region"].unique()):
    sub = region_year[region_year["region"]==r]
    plt.scatter(sub["hdi"], sub["log_attacks"], alpha=0.6, label=r)
plt.xlabel("median HDI (region-year)")
plt.ylabel("median log_attacks")
plt.title("Region-year: HDI vs log_attacks")
plt.legend(bbox_to_anchor=(1.05,1), loc="upper left")
plt.show()


In [ ]:
# 1.5 Correlation (simple, not causal)
corr = df[["hdi","log_attacks","log_casualties","log_casualties_per_attack"]].corr(numeric_only=True)
display(corr)


## 2) Modeling setup (time-aware split)

Because the data is yearly, we use **time-based split**:
- Train = earlier years
- Test  = later years

This helps avoid information leakage (future → past).


In [ ]:
# Choose cutoff year (edit if you want)
cutoff_year = 2012

# Feature set
feature_cols = ["hdi", "year", "country", "region", "hdi_group"]

# Targets we will model
TARGETS = ["log_attacks", "log_casualties", "log_casualties_per_attack"]

# Prepare split once
data_all = df[feature_cols + TARGETS].dropna().copy()
train = data_all[data_all["year"] <= cutoff_year].copy()
test  = data_all[data_all["year"] >  cutoff_year].copy()

print("Train:", train.shape, "years", train["year"].min(), "-", train["year"].max())
print("Test :", test.shape,  "years", test["year"].min(),  "-", test["year"].max())


## 3) Pipelines + evaluation metrics

We build a single preprocessing transformer:
- numeric: `hdi`, `year` (scaled)
- categorical: `country`, `region`, `hdi_group` (one-hot)

Then we fit 3 models:
- Linear Regression (baseline)
- Decision Tree (tuned)
- Random Forest (tuned)

Metrics:
- **MAE**
- **RMSE**
- **R²**


In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

cat_cols = ["country","region","hdi_group"]
num_cols = ["hdi","year"]

preprocess = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), num_cols),
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
    ]
)

def eval_reg(y_true, y_pred):
    return {
        "MAE":  mean_absolute_error(y_true, y_pred),
        "RMSE": mean_squared_error(y_true, y_pred, squared=False),
        "R2":   r2_score(y_true, y_pred),
    }


In [ ]:
# Modeling function (returns fitted models + results row)
def fit_models_for_target(target, X_train, y_train, X_test, y_test):
    rows = []
    fitted = {}

    # 1) Linear Regression
    lin = Pipeline([("prep", preprocess), ("model", LinearRegression())])
    lin.fit(X_train, y_train)
    pred = lin.predict(X_test)
    rows.append({"target": target, "model": "LinearRegression", **eval_reg(y_test, pred)})
    fitted["LinearRegression"] = lin

    # 2) Decision Tree (tuned - small grid to keep runtime reasonable)
    tree = Pipeline([("prep", preprocess), ("model", DecisionTreeRegressor(random_state=42))])
    grid_tree = {
        "model__max_depth": [3, 5, 8, None],
        "model__min_samples_leaf": [1, 2, 5, 10],
        "model__min_samples_split": [2, 5, 10],
    }
    gs_tree = GridSearchCV(tree, grid_tree, cv=5, scoring="neg_mean_squared_error", n_jobs=-1)
    gs_tree.fit(X_train, y_train)
    best_tree = gs_tree.best_estimator_
    pred = best_tree.predict(X_test)
    rows.append({"target": target, "model": f"DecisionTree(tuned)", **eval_reg(y_test, pred),
                 "best_params": gs_tree.best_params_})
    fitted["DecisionTree"] = best_tree

    # 3) Random Forest (tuned - small grid)
    rf = Pipeline([("prep", preprocess), ("model", RandomForestRegressor(random_state=42, n_jobs=-1))])
    grid_rf = {
        "model__n_estimators": [200, 400],
        "model__max_depth": [None, 10, 20],
        "model__min_samples_leaf": [1, 2, 5],
    }
    gs_rf = GridSearchCV(rf, grid_rf, cv=5, scoring="neg_mean_squared_error", n_jobs=-1)
    gs_rf.fit(X_train, y_train)
    best_rf = gs_rf.best_estimator_
    pred = best_rf.predict(X_test)
    rows.append({"target": target, "model": f"RandomForest(tuned)", **eval_reg(y_test, pred),
                 "best_params": gs_rf.best_params_})
    fitted["RandomForest"] = best_rf

    return rows, fitted


## 4) Run models for all targets

In [ ]:
results_rows = []
all_fitted = {}

X_train = train[feature_cols]
X_test  = test[feature_cols]

for target in TARGETS:
    y_train = train[target]
    y_test  = test[target]
    rows, fitted = fit_models_for_target(target, X_train, y_train, X_test, y_test)
    results_rows.extend(rows)
    all_fitted[target] = fitted

results = pd.DataFrame(results_rows)
display(results.sort_values(["target","RMSE"]))


## 5) Diagnostics: residuals vs HDI (for the best model per target)

If residuals show patterns across HDI, it suggests the model still misses structure.


In [ ]:
# Pick best model by lowest RMSE per target
best_by_target = (results.sort_values("RMSE")
                  .groupby("target", as_index=False)
                  .first()[["target","model"]])
display(best_by_target)


In [ ]:
def get_model_obj(target, model_name):
    # normalize labels
    if "Linear" in model_name:
        return all_fitted[target]["LinearRegression"]
    if "DecisionTree" in model_name:
        return all_fitted[target]["DecisionTree"]
    return all_fitted[target]["RandomForest"]

for _, row in best_by_target.iterrows():
    target = row["target"]
    model_name = row["model"]
    model = get_model_obj(target, model_name)
    y_test = test[target]
    pred = model.predict(X_test)
    residuals = y_test.values - pred

    plt.figure()
    plt.scatter(X_test["hdi"], residuals, alpha=0.5)
    plt.axhline(0)
    plt.xlabel("HDI")
    plt.ylabel("Residual (true - pred)")
    plt.title(f"Residuals vs HDI | target={target} | model={model_name}")
    plt.show()


## 6) Feature importance (Random Forest)

One-hot encoding creates many features (country/region dummies).
We print **top 20** important features for each target's random forest model.

> If `hdi` is consistently among top features, that's evidence HDI contributes predictive signal **after controlling for** region/country/year.


In [ ]:
import numpy as np

def rf_feature_importance_table(rf_pipeline, top_n=20):
    ohe = rf_pipeline.named_steps["prep"].named_transformers_["cat"]
    cat_names = ohe.get_feature_names_out(cat_cols)
    feature_names = np.concatenate([num_cols, cat_names])
    importances = rf_pipeline.named_steps["model"].feature_importances_
    fi = (pd.DataFrame({"feature": feature_names, "importance": importances})
            .sort_values("importance", ascending=False)
            .head(top_n))
    return fi

for target in TARGETS:
    rf_model = all_fitted[target]["RandomForest"]
    print("\n=== Target:", target, "===")
    display(rf_feature_importance_table(rf_model, top_n=20))


## 7) (Optional) “HDI effect” with a simple what-if curve

We hold `country/region/hdi_group` fixed and vary HDI to see the model’s predicted trend.

This is **not causal**, but it helps your discussion.


In [ ]:
def what_if_hdi_curve(target, country, region, hdi_group, year, model_kind="RandomForest"):
    model = all_fitted[target][model_kind]
    hdi_grid = np.linspace(df["hdi"].min(), df["hdi"].max(), 60)

    Xw = pd.DataFrame({
        "hdi": hdi_grid,
        "year": [year]*len(hdi_grid),
        "country": [country]*len(hdi_grid),
        "region": [region]*len(hdi_grid),
        "hdi_group": [hdi_group]*len(hdi_grid),
    })

    pred = model.predict(Xw)

    plt.figure()
    plt.plot(hdi_grid, pred)
    plt.xlabel("HDI")
    plt.ylabel(f"Predicted {target}")
    plt.title(f"What-if: {target} vs HDI | {model_kind} | {country}, {year}")
    plt.show()

# Example (edit freely)
what_if_hdi_curve(
    target="log_attacks",
    country="Turkey",
    region="Middle East & North Africa",
    hdi_group="High",
    year=2015,
    model_kind="RandomForest"
)


## 8) Save results to Excel/CSV (for your report)

You can paste tables + screenshots of plots to your report, but it's nice to also export the metrics.


In [ ]:
out_csv = "model_results.csv"
results.to_csv(out_csv, index=False)
print("Saved:", out_csv)


# Report checklist (copy into your write-up)

- **Research question:** Is HDI associated with terrorism frequency/severity?
- **Targets:** log_attacks, log_casualties, log_casualties_per_attack
- **Why log transform:** heavy tails/outliers
- **Train/test split:** time-based (train <= cutoff_year, test > cutoff_year)
- **Models:** Linear Regression, tuned Decision Tree, tuned Random Forest
- **Metrics:** MAE/RMSE/R² (compare models)
- **Interpretation:**
  - Feature importance (is HDI important after controls?)
  - Residual vs HDI plot (systematic patterns?)
  - Optional what-if HDI curve (discussion aid)
- **Limitations:** correlation vs causality; missing confounders; measurement errors; aggregation by country-year
